In [1]:
#importing library
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns

**TRATAMENTO DOS DADOS**

In [2]:
#loading the dataset
file_path = '/content/drive/MyDrive/Colab Notebooks/bank-additional-full.csv'
df=pd.read_csv(file_path,delimiter=';')

#showing the datatset
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [4]:
# describe estatistic
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [3]:
# substituindo '.' por '_' nas colunas
df.columns = df.columns.str.replace(".", "_")

In [5]:
df.describe()

,age,duration,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed
count,41188.00000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000,41188.000000
mean,40.02406,258.285010,2.567593,962.475454,0.172963,0.081886,93.575664,-40.502600,3.621291,5167.035911
std,10.42125,259.279249,2.770014,186.910907,0.494901,1.570960,0.578840,4.628198,1.734447,72.251528
min,17.00000,0.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.00000,102.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.344000,5099.100000
50%,38.00000,180.000000,2.000000,999.000000,0.000000,1.100000,93.749000,-41.800000,4.857000,5191.000000
75%,47.00000,319.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.00000,4918.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [ ]:
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns
for feature in numeric_features:
    sns.boxplot(x=df[feature])
    plt.title(f'Boxplot of {feature}')
    plt.show()


In [5]:
# nossa variavel de interesse (y) indica se o usuario aceitou ou nao a oferta da campanha
# transformando em booleano
df['y'] = df['y'].map({'yes': 1, 'no': 0}) # the map function identify the values in the column and replaces them

In [6]:
df['y'].value_counts()

,count
y,
0,36548
1,4640


In [40]:
df.isnull().sum()

,0
age,0
job,0
marital,0
education,0
default,0
housing,0
loan,0
contact,0
month,0
day_of_week,0


In [9]:
df['pdays'].unique()

array([999,   6,   4,   3,   5,   1,   0,  10,   7,   8,   9,  11,   2,
        12,  13,  14,  15,  16,  21,  17,  18,  22,  25,  26,  19,  27,
        20])

In [6]:
# transformando 999 em 0 já que nao houve contato
df['pdays'] = df['pdays'].replace({999:0})

In [7]:
#função que verifica se houve contato antes
def is_contacted_before(x):
    if x == 0:
        return '0'
    else:
        return '1'

In [8]:
df['contacted_before'] = df['pdays'].apply(is_contacted_before)
df['contacted_before'].value_counts(normalize=True)*100 #
df['contacted_before'] = df['contacted_before'].astype(int)

In [9]:
df.replace('unknown', np.nan, inplace=True)
print(df.isna().sum()/len(df)*100)

age                  0.000000
job                  0.801204
marital              0.194231
education            4.202680
default             20.872584
housing              2.403613
loan                 2.403613
contact              0.000000
month                0.000000
day_of_week          0.000000
duration             0.000000
campaign             0.000000
pdays                0.000000
previous             0.000000
poutcome             0.000000
emp_var_rate         0.000000
cons_price_idx       0.000000
cons_conf_idx        0.000000
euribor3m            0.000000
nr_employed          0.000000
y                    0.000000
contacted_before     0.000000
dtype: float64


In [45]:
df["default"].value_counts(normalize=True)*100

,proportion
default,
no,99.990795
yes,0.009205


In [15]:
print(df.isna().sum())

age                    0
job                  330
marital               80
education           1731
default             8597
housing              990
loan                 990
contact                0
month                  0
day_of_week            0
duration               0
campaign               0
pdays                  0
previous               0
poutcome               0
emp_var_rate           0
cons_price_idx         0
cons_conf_idx          0
euribor3m              0
nr_employed            0
y                      0
contacted_before       0
dtype: int64


In [10]:
# como a variavel default apresenta uma variância muito pequena com muitos valores nulos, vamos removê-la
df_cleaned = df.drop('default', axis=1)

In [11]:
# tratando missing values: como temos valores nulos para variáveis categóricas, optei por imputar a moda já que árvores não são muito sensíveis a essa metodologia
for column in ['job', 'marital', 'education', 'housing', 'loan']:
    df_cleaned[column] = df_cleaned[column].fillna(df_cleaned[column].mode()[0])

In [12]:
# checando os tratamentos de nulls
print(df_cleaned.isna().sum())

age                 0
job                 0
marital             0
education           0
housing             0
loan                0
contact             0
month               0
day_of_week         0
duration            0
campaign            0
pdays               0
previous            0
poutcome            0
emp_var_rate        0
cons_price_idx      0
cons_conf_idx       0
euribor3m           0
nr_employed         0
y                   0
contacted_before    0
dtype: int64


In [13]:
def onehot_encode(df,columns):
    df=df.copy()
    for column in columns:
        dummies=pd.get_dummies(df[column],prefix=column, drop_first=True).astype(int)
        df=pd.concat([df,dummies],axis=1)
        df=df.drop(column,axis=1)
    return df

In [14]:
nominal_feature=['job','marital','education','poutcome', 'contact']

binary_feature=['housing','loan']

ordinal_feature=['day_of_week','month']

In [16]:
# verificando os valores das colunas ordinárias
for column in ordinal_feature:
    values = df_cleaned[column].unique()
    print(values)

['mon' 'tue' 'wed' 'thu' 'fri']
['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'mar' 'apr' 'sep']


In [15]:
# transformando variáveis ordinais em valores numéricos
day_mapping = {
    'mon': 1,
    'tue': 2,
    'wed': 3,
    'thu': 4,
    'fri': 5
}
df_cleaned['day_of_week'] = df_cleaned['day_of_week'].map(day_mapping)

month_mapping = {
    'jan': 1, 'feb': 2, 'mar': 3, 'apr': 4,
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8,
    'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12
}
df_cleaned['month'] = df_cleaned['month'].map(month_mapping)

In [16]:
# transformando as colunas booleanas em 0 e 1
for column in ['housing','loan']:
    df_cleaned[column]=df_cleaned[column].map({'yes':1,'no':0})

In [19]:
df_cleaned['contact'].value_counts(normalize=True)*100

,proportion
contact,
cellular,63.474798
telephone,36.525202


In [17]:
# dummizando as features categóricas
df_cleaned_dummie = onehot_encode(df_cleaned,nominal_feature)

In [21]:
df_cleaned_dummie.dtypes

,0
age,int64
housing,int64
loan,int64
month,int64
day_of_week,int64
duration,int64
campaign,int64
pdays,int64
previous,int64
emp_var_rate,float64


**SEPARAÇÃO DO CONJUNTO DE DADOS EM TREINO E TESTE**

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, roc_auc_score, precision_recall_curve, f1_score, confusion_matrix, classification_report, precision_score, recall_score

In [19]:
X = df_cleaned_dummie.drop(columns = ['y'])
y = df_cleaned_dummie['y']

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

**MODELAGEM**

XGBoost


In [26]:
import xgboost as xgb
import sklearn

print(xgb.__version__)  # Deve ser >= 1.6.0
print(sklearn.__version__)  # Deve ser >= 1.0.0


2.1.3
1.6.0


In [27]:
pip install --upgrade scikit-learn xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 87.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [28]:
pip install scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 50.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires scikit-learn<2,>=1.3.2, but you have scikit-learn 1.2.2 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.


In [21]:
import xgboost as xgb
from xgboost import XGBClassifier

In [22]:
# Treinar um classificador XGBoost
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42) #to com dificuldade de saber como ajustar os parâmetros para tentar melhorar o desempenho
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [23]:
# Avaliar o modelo
y_pred = xgb_model.predict(X_test)

In [24]:
# Acurácia do Modelo
print("Acurácia do CatBoost:", accuracy_score(y_test, y_pred), "\n")

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:\n", cm, "\n")

# Previsão de probabilidades para AUC-ROC
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# AUC-ROC
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc_roc, "\n")

# Métricas adicionais: Precision, Recall e F1-Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precisão: {precision}") # proporção de previsões positivas corretas (tp/tp+fp)
print(f"(Recall): {recall}") # taxa de acerto do evento (tp/tp+fn)
print(f"F1-Score: {f1} \n")

# Relatório de Classificação
report = classification_report(y_test, y_pred)
print("Relatório de Classificação:\n", report)

Acurácia do CatBoost: 0.9163227320547058 

Matriz de Confusão:
 [[10641   327]
 [  707   682]] 

AUC-ROC: 0.9486298645342508 

Precisão: 0.6759167492566898
(Recall): 0.49100071994240463
F1-Score: 0.5688073394495413 

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95     10968
           1       0.68      0.49      0.57      1389

    accuracy                           0.92     12357
   macro avg       0.81      0.73      0.76     12357
weighted avg       0.91      0.92      0.91     12357



AUC-ROC indica que houve uma boa distinção de classes, o que é legal pro problema de negócio.
Acredito que dentro de uma campanha o foco principal é acertar clientes que tenham potencial de conversão, pois perder um pouco com a comunicação de quem nao aceite nao gera tanto impacto que deixar de converter um cliente potêncial. Por isso, o baixo índice de recall pode ser um indicativo que o threshold padrão de 0.5 não ta capturando bem todos os eventos. Ajustá-lo pode melhorar o modelo para atender melhor o objetivo do negócio, mas antes vamos verificar qual modelo foi melhor e considerar essas métricas com maior importância.

**CatBoost**

In [26]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.0 MB/s eta 0:00:00


In [27]:
from catboost import CatBoostClassifier

In [28]:
# Treinar um classificador CatBoost
cat_model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0)
cat_model.fit(X_train, y_train)

In [29]:
# Avaliar o modelo
y_pred = cat_model.predict(X_test)

In [30]:
# Acurácia do Modelo
print("Acurácia do CatBoost:", accuracy_score(y_test, y_pred), "\n")

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:\n", cm, "\n")

# Previsão de probabilidades para AUC-ROC
y_pred_proba = cat_model.predict_proba(X_test)[:, 1]

# AUC-ROC
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc_roc, "\n")

# Métricas adicionais: Precision, Recall e F1-Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precisão: {precision}") # proporção de previsões positivas corretas (tp/tp+fp)
print(f"(Recall): {recall}") # taxa de acerto do evento (tp/tp+fn)
print(f"F1-Score: {f1} \n")

# Relatório de Classificação
report = classification_report(y_test, y_pred)
print("Relatório de Classificação:\n", report)

Acurácia do CatBoost: 0.9173747673383508 

Matriz de Confusão:
 [[10622   346]
 [  675   714]] 

AUC-ROC: 0.9500880629768438 

Precisão: 0.6735849056603773
(Recall): 0.5140388768898488
F1-Score: 0.5830951408738261 

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.94      0.97      0.95     10968
           1       0.67      0.51      0.58      1389

    accuracy                           0.92     12357
   macro avg       0.81      0.74      0.77     12357
weighted avg       0.91      0.92      0.91     12357



O Catboost apresentou melhores métricas que o XGboost entretanto ele ainda não se mostra eficiente para prever o evento (aceitantes da oferta) o que para o cenário de negócio se torna um problema. A tratativa de diminuir o threshold ainda se mostra interessante.

**LightGBM**

In [32]:
!pip install lightgbm

In [33]:
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [34]:
# Treinar um classificador LightGBM
lgb_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, num_leaves=31, random_state=42)
lgb_model.fit(X_train, y_train)

[LightGBM] [Info] Number of positive: 3251, number of negative: 25580
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.039072 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 737
[LightGBM] [Info] Number of data points in the train set: 28831, number of used features: 35
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.112761 -> initscore=-2.062848
[LightGBM] [Info] Start training from score -2.062848


LGBMClassifier(random_state=42)

In [35]:
# Avaliar o modelo
y_pred = lgb_model.predict(X_test)

In [36]:
# Acurácia do Modelo
print("Acurácia do LightGBM:", accuracy_score(y_test, y_pred), "\n")

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão:\n", cm, "\n")

# Previsão de probabilidades para AUC-ROC
y_pred_proba = lgb_model.predict_proba(X_test)[:, 1]

# AUC-ROC
auc_roc = roc_auc_score(y_test, y_pred_proba)
print("AUC-ROC:", auc_roc, "\n")

# Métricas adicionais: Precision, Recall e F1-Score
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precisão: {precision}") # proporção de previsões positivas corretas (tp/tp+fp)
print(f"(Recall): {recall}") # taxa de acerto do evento (tp/tp+fn)
print(f"F1-Score: {f1} \n")

# Relatório de Classificação
report = classification_report(y_test, y_pred)
print("Relatório de Classificação:\n", report)

Acurácia do LightGBM: 0.9162418062636563 

Matriz de Confusão:
 [[10582   386]
 [  649   740]] 

AUC-ROC: 0.9505487919828558 

Precisão: 0.6571936056838366
(Recall): 0.5327573794096472
F1-Score: 0.588469184890656 

Relatório de Classificação:
               precision    recall  f1-score   support

           0       0.94      0.96      0.95     10968
           1       0.66      0.53      0.59      1389

    accuracy                           0.92     12357
   macro avg       0.80      0.75      0.77     12357
weighted avg       0.91      0.92      0.91     12357



A acurácia do catboost foi ligeiramente superior a do LGBM, entretanto, tivemos um recall superior no LGBM o que para o problema de negócio é mais interessante prever quem converteu. De todo modo, ainda é um recall baixo por isso vou tentar diminuir o trashold no modelo que desempenhou melhor para tentar contornar o problema.

**Reduzindo o Threshold no LightGBM para tentar melhor a previsão de TP.**

In [39]:
# Definir um novo threshold
threshold = 0.3  # Tente um valor menor que o padrão (0.5)

# Acurácia do Modelo
print("Acurácia do LightGBM:", accuracy_score(y_test, y_pred), "\n")

# Ajustar as previsões com o novo threshold
y_pred_adjusted = (y_pred_proba >= threshold).astype(int)

# Recalcular as métricas com o novo threshold
precision = precision_score(y_test, y_pred_adjusted)
recall = recall_score(y_test, y_pred_adjusted)
f1 = f1_score(y_test, y_pred_adjusted)

# Matriz de Confusão
cm = confusion_matrix(y_test, y_pred_adjusted)

# Exibir resultados
print("Novo Threshold:", threshold)
print("Matriz de Confusão:\n", cm)
print(f"Precisão Ajustada: {precision}")
print(f"Recall Ajustado: {recall}")
print(f"F1-Score Ajustado: {f1}")

Acurácia do LightGBM: 0.9162418062636563 

Novo Threshold: 0.3
Matriz de Confusão:
 [[10126   842]
 [  302  1087]]
Precisão Ajustada: 0.5635044064282011
Recall Ajustado: 0.7825773938084953
F1-Score Ajustado: 0.6552139843279084


**Utilizando Pycaret**

In [ ]:
# Instalar a biblioteca (caso não esteja instalada)
!pip install pycaret[full]

In [1]:
# Importar as bibliotecas necessárias
from pycaret.classification import *

In [28]:
# Carregar um conjunto de dados de exemplo
#from pycaret.datasets import get_data
#data = get_data('titanic')  # Dataset de sobreviventes do Titanic

# Configurar o ambiente do PyCaret
setup(
    data=df_cleaned,
    target='y',  # Coluna de alvo
    session_id=42,      # Reprodutibilidade
    categorical_features= ['housing','loan','contact']  # Identificar colunas categóricas
)

# Comparar modelos
best_model = compare_models()

# Mostrar o melhor modelo
print("Melhor modelo:", best_model)

,Description,Value
0,Session id,42
1,Target,y
2,Target type,Binary
3,Original data shape,"(41188, 21)"
4,Transformed data shape,"(41188, 21)"
5,Transformed train set shape,"(28831, 21)"
6,Transformed test set shape,"(12357, 21)"
7,Numeric features,13
8,Categorical features,3
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9154,0.9486,0.5413,0.6498,0.5904,0.5437,0.5467,1.4700
dummy,Dummy Classifier,0.8873,0.5000,0.0000,0.0000,0.0000,0.0000,0.0000,0.1580


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

Melhor modelo: LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)
